<a href="https://colab.research.google.com/github/Diptesh2006/smart-waste-classifier/blob/main/CSIVIT_Garbage_Forkthis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, glob, shutil, zipfile, random, math
from collections import Counter

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

from google.colab import files


In [ ]:

print(" upload your dataset .zip file ")
uploaded = files.upload()

zip_path = list(uploaded.keys())[0]
with zipfile.ZipFile(zip_path, 'r') as zf:
    zf.extractall("/content/trashnet")

print("Dataset at /content/trashnet")


 upload your dataset .zip file 


Saving archive (10).zip to archive (10) (1).zip
Dataset at /content/trashnet


In [ ]:


BASE = "/content/trashnet/dataset-resized"


paths = sum([glob.glob(os.path.join(BASE, c, "*.jpg"))
             for c in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, c))], [])
labels_str = [os.path.basename(os.path.dirname(p)) for p in paths]


In [ ]:
X_tr, X_val, y_tr_str, y_val_str = train_test_split(
    paths, labels_str, test_size=0.2, stratify=labels_str, random_state=42
)


In [ ]:
class_names   = sorted(set(labels_str))
class_to_idx  = {c:i for i,c in enumerate(class_names)}
idx_to_class  = {i:c for c,i in class_to_idx.items()}
y_tr  = np.array([class_to_idx[s] for s in y_tr_str],  dtype=np.int32)
y_val = np.array([class_to_idx[s] for s in y_val_str], dtype=np.int32)

In [ ]:
print("Classes:", class_names)
print("Train/Val:", len(X_tr), len(X_val))

Classes: ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
Train/Val: 2021 506


In [ ]:
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32

In [ ]:
def decode_and_resize(path, y):
    x = tf.io.decode_image(tf.io.read_file(path), channels=3, expand_animations=False)
    x = tf.image.convert_image_dtype(x, tf.float32)         # [0,1]
    x = tf.image.resize(x, IMAGE_SIZE, antialias=True)
    return x, y

In [ ]:
augment = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.10),
    layers.RandomTranslation(0.05, 0.05),
])

In [ ]:
AUTOTUNE   = tf.data.AUTOTUNE

In [ ]:
def make_ds(X, y, augment_on=False, shuffle=False):
    ds = tf.data.Dataset.from_tensor_slices((X, y))
    if shuffle: ds = ds.shuffle(len(X), seed=42, reshuffle_each_iteration=True)
    ds = ds.map(decode_and_resize, num_parallel_calls=AUTOTUNE)
    if augment_on:
        ds = ds.map(lambda x, y: (augment(x, training=True), y), num_parallel_calls=AUTOTUNE)
    return ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)

In [ ]:
train_ds = make_ds(X_tr,  y_tr,  augment_on=True,  shuffle=True)
val_ds   = make_ds(X_val, y_val, augment_on=False, shuffle=False)

In [ ]:
# weights[i] = N / (K * count_i)
counts  = np.bincount(y_tr, minlength=len(class_names))
weights = {i: (len(y_tr) / (len(class_names) * counts[i])) for i in range(len(class_names))}
weights


{0: np.float64(1.0460662525879918),
 1: np.float64(0.8399833748960931),
 2: np.float64(1.026930894308943),
 3: np.float64(0.7091228070175438),
 4: np.float64(0.8748917748917749),
 5: np.float64(3.062121212121212)}

In [ ]:
def build_baseline(num_classes):
    inp = layers.Input((*IMAGE_SIZE, 3))
    x = layers.Conv2D(32, 3, padding='same', activation='relu')(inp); x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x);  x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128,3, padding='same', activation='relu')(x);  x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.30)(x)
    out = layers.Dense(num_classes, activation='softmax')(x)
    return keras.Model(inp, out, name="baseline_cnn")

model = build_baseline(len(class_names))
model.compile(optimizer=keras.optimizers.Adam(1e-3),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

cbs = [
    keras.callbacks.EarlyStopping(patience=4, restore_best_weights=True, monitor="val_accuracy"),
    keras.callbacks.ReduceLROnPlateau(patience=2, factor=0.5, min_lr=1e-6),
]

history = model.fit(train_ds, validation_data=val_ds, epochs=12,
                    class_weight=weights, callbacks=cbs, verbose=1)


Epoch 1/12
64/64 ━━━━━━━━━━━━━━━━━━━━ 281s 4s/step - accuracy: 0.1902 - loss: 1.8080 - val_accuracy: 0.3498 - val_loss: 1.5729 - learning_rate: 0.0010
Epoch 2/12
64/64 ━━━━━━━━━━━━━━━━━━━━ 284s 4s/step - accuracy: 0.3154 - loss: 1.5893 - val_accuracy: 0.2806 - val_loss: 1.6405 - learning_rate: 0.0010
Epoch 3/12
64/64 ━━━━━━━━━━━━━━━━━━━━ 317s 4s/step - accuracy: 0.3184 - loss: 1.5880 - val_accuracy: 0.3458 - val_loss: 1.5322 - learning_rate: 0.0010
Epoch 4/12
64/64 ━━━━━━━━━━━━━━━━━━━━ 327s 4s/step - accuracy: 0.3171 - loss: 1.5711 - val_accuracy: 0.3043 - val_loss: 1.6580 - learning_rate: 0.0010
Epoch 5/12
64/64 ━━━━━━━━━━━━━━━━━━━━ 313s 4s/step - accuracy: 0.3334 - loss: 1.5362 - val_accuracy: 0.3953 - val_loss: 1.4752 - learning_rate: 0.0010
Epoch 6/12
64/64 ━━━━━━━━━━━━━━━━━━━━ 274s 4s/step - accuracy: 0.3646 - loss: 1.4932 - val_accuracy: 0.4209 - val_loss: 1.3980 - learning_rate: 0.0010
Epoch 7/12
64/64 ━━━━━━━━━━━━━━━━━━━━ 275s 4s/step - accuracy: 0.4174 - loss: 1.3960 - val_acc